In [36]:
####################   llamma  #######################

import json
import boto3
from botocore.exceptions import ClientError

def get_response(formatted_prompt):
    # Create a Bedrock Runtime client in the AWS Region of your choice.
    client = boto3.client("bedrock-runtime", region_name="eu-west-1")

    # In case of llamma 3 only inference model ids are supported. so we are using "eu."
    model_id = "eu.meta.llama3-2-3b-instruct-v1:0"

    # Format the request payload using the model's native structure.
    native_request = {
        "prompt": formatted_prompt,
        "max_gen_len": 512,
        "temperature": 0.5,
    }

    # Convert the native request to JSON.
    request = json.dumps(native_request)

    try:
        # Invoke the model with the request.
        response = client.invoke_model(modelId=model_id, body=request)
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        return None

    # Decode the response body.
    model_response = json.loads(response["body"].read())

    # Extract and return the response text.
    response_text = model_response["generation"]
    return response_text

prompt= """ 
what is the capital of india. give ouput in json format, assistant:
"""

formatted_prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
 {prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""
response_text = get_response(formatted_prompt)
print(response_text)

{
  "capital": "New Delhi"
}


In [38]:
# Use the native inference API to send a text message to Mistral.

import boto3
import json
from botocore.exceptions import ClientError


def get_response_mistral(formatted_prompt):
    # Create a Bedrock Runtime client in the AWS Region of your choice.
    client = boto3.client("bedrock-runtime", region_name="eu-west-1")

    model_id = "mistral.mistral-large-2402-v1:0"

    # Format the request payload using the model's native structure.
    native_request = {
        "prompt": formatted_prompt,
        "max_tokens": 512,
        "temperature": 0.5,
    }

    # Convert the native request to JSON.
    request = json.dumps(native_request)

    try:
        # Invoke the model with the request.
        response = client.invoke_model(modelId=model_id, body=request)
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        return None

    # Decode the response body.
    model_response = json.loads(response["body"].read())
    # Extract and return the response text.
    response_text = model_response["outputs"][0]["text"]
    return response_text


prompt= """ 
what is the capital of india. give ouput in json format, assistant:
"""

# Embed the prompt in Mistral's instruction format.
formatted_prompt = f"""<s>[INST] {prompt} [/INST]"""

response_text = get_response_mistral(formatted_prompt)
print(response_text)

 {
    "Question": "What is the capital of India?",
    "Answer": "New Delhi"
  }


In [50]:
# Use the native inference API to send a text message to claude.

import boto3
import json
from botocore.exceptions import ClientError

def question_answer_claude(prompt):
    client = boto3.client("bedrock-runtime", region_name= 'eu-west-1')
    
    model_id = "eu.anthropic.claude-3-5-sonnet-20240620-v1:0"
    
    # Format the request payload using the model's native structure.
    native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "temperature": 0.5,
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": prompt}],
            }
        ],
    }
    # Convert the native request to JSON.
    request = json.dumps(native_request)
    try:
        # Invoke the model with the request.
        response = client.invoke_model(modelId=model_id, body=request)
        model_response = json.loads(response["body"].read())
        response_text = model_response["content"][0]["text"]
        return response_text
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        raise


prompt= """ 
what is the capital of india. give ouput in json format, assistant:
"""

response_text = question_answer_claude(prompt)
print(response_text)

Here is the answer in JSON format:

{
  "question": "What is the capital of India?",
  "answer": "New Delhi",
  "country": "India",
  "capital": "New Delhi",
  "type": "National Capital",
  "location": {
    "latitude": 28.6139,
    "longitude": 77.2090
  }
}
